In [1]:
from itertools import product

import pandas as pd
import numpy as np
import scipy.io as scio
from sklearn.svm import SVC

%load_ext autoreload
%autoreload 2
import main

In [6]:
mat = scio.loadmat('spamTrain.mat')

# TODO?: normalize
x = pd.DataFrame(mat['X'])
y = pd.DataFrame(mat['y']).iloc[:, 0]

mat = scio.loadmat('spamTest.mat')
xtest = pd.DataFrame(mat['Xtest'])
ytest = pd.DataFrame(mat['ytest']).iloc[:, 0]

In [136]:
C = np.logspace(-1, 1.5, 3)
gammas = np.logspace(-3, 1, 3)

params = pd.DataFrame(product(C, gammas), columns=['C', 'gamma'])

In [137]:
params['score'] = 0

for i in range(len(params)):
    clf = SVC(kernel='rbf', C=params.iloc[i].C, gamma=params.iloc[i].gamma)
    clf.fit(x, y)
    # NOTE: we shouldn't use test sample for validation!
    score = clf.score(xtest, ytest)
    params.iloc[i, 2] = score

index = pd.Series(params.score.idxmax()).iloc[0]
c = params.C[index]
gamma = params.gamma[index]
print('Best: c={}, gamma={}'.format(c, gamma))

Best: c=31.622776601683793, gamma=0.001


In [146]:
def get_features(text, vocab):
    result = np.zeros(len(words))
    for word in main.get_words(text):
        if word in vocab:
            result[vocab[word]] = 1
    return result

filenames = ['emailSample1.txt', 'emailSample2.txt', 'spamSample1.txt', 'spamSample2.txt']
texts = []

for filename in filenames:
    with open(filename, 'r') as f:
        texts.append(f.read())

In [147]:
words = pd.read_csv('vocab.txt', sep='\t', header=None, names=['code', 'word'])
vocab = {word: code for code, word in words.itertuples(index=False)}
x_emails = pd.DataFrame([get_features(t, vocab) for t in texts])

clf = SVC(kernel='rbf', C=c, gamma=gamma)
clf.fit(x, y)
prediction = clf.predict(x_emails)

for i in range(len(x_emails)):
    if prediction[i] == 1:
        print('{} is spam'.format(filenames[i]))
    else:
        print('{} is not spam'.format(filenames[i]))

emailSample1.txt is not spam
emailSample2.txt is not spam
spamSample1.txt is not spam
spamSample2.txt is spam


In [172]:
df = pd.read_csv('train.csv')
x_custom = df.iloc[:, :-2]
y_custom = df.iloc[:, -1]

words = pd.read_csv('custom.txt', header=None)
vocab = {word: index for index, word in words.itertuples()}
x_emails = pd.DataFrame([get_features(t, vocab) for t in texts])
print(x_emails.iloc[2].sum())

clf = SVC(kernel='rbf', C=1, gamma='auto')
clf.fit(x_custom, y_custom)
prediction = clf.predict(x_emails)

for i in range(len(x_emails)):
    if prediction[i] == 1:
        print('{} is spam'.format(filenames[i]))
    else:
        print('{} is not spam'.format(filenames[i]))

52.0
emailSample1.txt is not spam
emailSample2.txt is not spam
spamSample1.txt is not spam
spamSample2.txt is not spam
